In [1]:
import os
print('os.getcwd()', os.getcwd())
import sys
sys.path.insert(1, '../')
print(sys.version)
import time

#plotting tools
from matplotlib import pyplot as plt 
from tqdm.notebook import tqdm as tqdm

#torch libs
import torch
print('torch.__version__', torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe_device = 0 if torch.cuda.is_available() else -1
print(pipe_device, device)

#huggingface transformers
import transformers
print('transformers.__version__',transformers.__version__)
from transformers import AutoTokenizer, pipeline
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2PreTrainedModel

from datasets import load_dataset

#curious
from curious.models import GPT2HeadWithValueModel
from curious.rl import PPOTrainer
from curious.utils import LengthSampler, collater, respond_to_batch, generate_text

#jupyter stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

os.getcwd() /home/carson/Desktop/Projects/language_reinforce/notebooks
3.7.5 (default, Dec  9 2021, 17:04:37) 
[GCC 8.4.0]
torch.__version__ 1.12.1+cu102
0 cuda
transformers.__version__ 4.22.2


In [4]:
dataloader = torch.load('../small_data/dataloader.pth')

# the dataloader is just to seed the rollout
next(iter(dataloader))['query'][:3]

['I rented I AM CURI', '"I Am Curious: Yellow"', 'If only to avoid']

In [8]:
sentiment_pipe = pipeline(
    "sentiment-analysis",
    "lvwerra/distilbert-imdb", 
    device = pipe_device,
)

# reward model configs
sent_kwargs = {
    "return_all_scores": True,
    "function_to_apply": "none",
    "batch_size": 16
}

text = ['this movie was really bad!!', 'i loved this movie']
sentiments = sentiment_pipe(text, **sent_kwargs)

sentiments

[[{'label': 'NEGATIVE', 'score': 2.3350484371185303},
  {'label': 'POSITIVE', 'score': -2.726576566696167}],
 [{'label': 'NEGATIVE', 'score': -2.3049933910369873},
  {'label': 'POSITIVE', 'score': 2.6404218673706055}]]

In [10]:
gpt2_tokenizer = AutoTokenizer.from_pretrained(
    'gpt2',
    pad_token='<|endoftext|>',
    padding_side = 'left',
)

In [9]:
#cache_folder = '/Users/carson/projects/modelstates' 
cache_folder = '/home/carson/Desktop/Projects/modelstates'
model_name = 'distilgpt2'#'gpt2' # 'gpt2-xl' # 'gpt2-large' #
cache_dir = os.path.join(cache_folder, model_name)

gpt2_model = GPT2HeadWithValueModel.from_pretrained(
    model_name,
    cache_dir=cache_dir
)
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained(
    model_name,
    cache_dir=cache_dir
)

# put models onto GPU
gpt2_model.to(device)
gpt2_model_ref.to(device)

print(gpt2_model.device)

Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'v_head.summary.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'v_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'v_head.summary.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'v_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infer

cuda:0


In [14]:
ppo_trainer = PPOTrainer(gpt2_model, gpt2_model_ref, gpt2_tokenizer)

total_ppo_epochs = 150

for epoch, batch in tqdm(zip(range(total_ppo_epochs),iter(dataloader)),total=total_ppo_epochs):
    
    ### get start sequence input tokens
    query_tensors = [torch.tensor(t).long().to(device) for t in batch["tokens"]]
    
    break

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]